In [2]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.9 MB/s eta 0:00:00


In [3]:
import os
os.makedirs("sample_statements", exist_ok=True)

In [4]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import date

def generate_sample_statement(bank_name, cardholder, last4, start_date, end_date, due_date, total_due, min_due):
    filename = f"sample_statements/{bank_name}_statement.pdf"
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    # Header
    c.setFont("Helvetica-Bold", 16)
    c.drawString(200, 750, f"{bank_name} Credit Card Statement")

    # Card details
    c.setFont("Helvetica", 12)
    c.drawString(50, 710, f"Cardholder Name: {cardholder}")
    c.drawString(50, 690, f"Card Number: **** **** **** {last4}")
    c.drawString(50, 670, f"Billing Period: {start_date} to {end_date}")
    c.drawString(50, 650, f"Payment Due Date: {due_date}")

    # Amounts
    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, 630, f"Total Amount Due: Rs. {total_due}")
    c.drawString(50, 610, f"Minimum Amount Due: Rs. {min_due}")

    # Transactions section
    c.setFont("Helvetica-Bold", 13)
    c.drawString(50, 580, "Transactions:")
    c.setFont("Helvetica", 11)
    transactions = [
        ("01/10/2024", "Amazon Purchase", "₹ 1,299.00"),
        ("05/10/2024", "Swiggy Order", "₹ 425.50"),
        ("09/10/2024", "Zomato Dining", "₹ 870.00"),
        ("14/10/2024", "Fuel Payment", "₹ 2,100.00"),
    ]
    y = 560
    for date_str, desc, amt in transactions:
        c.drawString(60, y, f"{date_str}    {desc:25}    {amt}")
        y -= 20

    # Footer
    c.setFont("Helvetica-Oblique", 10)
    c.drawString(50, 100, "Thank you for using your Credit Card responsibly!")

    c.save()
    print(f"Created: {filename}")

# Generate 5 banks' statements
generate_sample_statement("HDFC", "Riya Sharma", "1234", "01/10/2024", "31/10/2024", "10/11/2024", "15,400.00", "800.00")
generate_sample_statement("ICICI", "Karan Patel", "5678", "03/10/2024", "02/11/2024", "12/11/2024", "10,200.00", "500.00")
generate_sample_statement("SBI", "Sneha Nair", "9101", "10/10/2024", "09/11/2024", "18/11/2024", "8,950.00", "450.00")
generate_sample_statement("Axis", "Rohan Deshmukh", "1122", "15/09/2024", "14/10/2024", "23/10/2024", "12,000.00", "600.00")
generate_sample_statement("Kotak", "Ananya Mehta", "3344", "05/10/2024", "04/11/2024", "13/11/2024", "9,600.00", "500.00")


Created: sample_statements/HDFC_statement.pdf
Created: sample_statements/ICICI_statement.pdf
Created: sample_statements/SBI_statement.pdf
Created: sample_statements/Axis_statement.pdf
Created: sample_statements/Kotak_statement.pdf


In [5]:
!pip install pdfplumber tabula-py pandas
!apt-get install -y openjdk-11-jdk-headless
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ca-certificates-java java-common libpcsclite1 openjdk-11-jre-headless
Suggested packages:
  default-jre pcscd openjdk-11-demo openjdk-11-source libnss-mdns
  fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei | fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  ca-certificates-java java-common libpcsclite1 openjdk-11-

In [1]:
!python parser.py --folder sample_statements

python3: can't open file '/content/parser.py': [Errno 2] No such file or directory


In [6]:
# Step 2: Parser Code
import pdfplumber
import re
import os
import json
import pandas as pd

def extract_text_from_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def regex_first_match(patterns, text):
    for p in patterns:
        m = re.search(p, text, flags=re.IGNORECASE)
        if m:
            return m.group(1).strip()
    return None

def clean_amount(s):
    return s.replace(",", "").strip() if s else s

def extract_fields_from_text(text):
    data = {}
    data["cardholder_name"] = regex_first_match([r"Cardholder Name[:\-]?\s*([A-Za-z\s]+)"], text) or "N/A"
    data["card_last4"] = regex_first_match([r"Card Number[:\-]?\s*\*+\s*(\d{4})"], text) or "N/A"
    data["billing_period"] = regex_first_match([r"Billing Period[:\-]?\s*(\d{2}/\d{2}/\d{4}\s*to\s*\d{2}/\d{2}/\d{4})"], text) or "N/A"
    data["due_date"] = regex_first_match([r"Payment Due Date[:\-]?\s*(\d{2}/\d{2}/\d{4})"], text) or "N/A"
    data["total_amount_due"] = clean_amount(regex_first_match([r"Total Amount Due[:\-]?\s*Rs\.?\s*([\d,\.]+)"], text)) or "N/A"
    return data

def parse_all_statements(folder="sample_statements"):
    all_data = []
    for fname in os.listdir(folder):
        if not fname.endswith(".pdf"):
            continue
        path = os.path.join(folder, fname)
        text = extract_text_from_pdf(path)
        fields = extract_fields_from_text(text)
        fields["bank"] = fname.split("_")[0]
        all_data.append(fields)
        print(f"Parsed: {fname} ✅")
    df = pd.DataFrame(all_data)
    df.to_csv("credit_card_summary.csv", index=False)
    print("\nSaved output -> credit_card_summary.csv")
    return df

# Run the parser
df = parse_all_statements()
df


Parsed: Axis_statement.pdf ✅
Parsed: ICICI_statement.pdf ✅
Parsed: SBI_statement.pdf ✅
Parsed: HDFC_statement.pdf ✅
Parsed: Kotak_statement.pdf ✅

Saved output -> credit_card_summary.csv


,cardholder_name,card_last4,billing_period,due_date,total_amount_due,bank
0,Rohan Deshmukh\nCard Number,N/A,15/09/2024 to 14/10/2024,23/10/2024,12000.00,Axis
1,Karan Patel\nCard Number,N/A,03/10/2024 to 02/11/2024,12/11/2024,10200.00,ICICI
2,Sneha Nair\nCard Number,N/A,10/10/2024 to 09/11/2024,18/11/2024,8950.00,SBI
3,Riya Sharma\nCard Number,N/A,01/10/2024 to 31/10/2024,10/11/2024,15400.00,HDFC
4,Ananya Mehta\nCard Number,N/A,05/10/2024 to 04/11/2024,13/11/2024,9600.00,Kotak
